# Link to the site
(https://www.reddit.com/r/MachineLearning/comments/8aimei/d_what_is_the_right_way_to_parallelize_rollouts/)

In [4]:
import gym, time, random, ray
import numpy as np
from gym import envs
from IPython.display import clear_output
import pandas as pd
env = gym.make('CartPole-v0') # try for different environements
# env = gym.make('FrozenLake-v0') # try for different environements
# env = gym.make('Robotank-ramNoFrameskip-v0') # try for different environements
ray.shutdown()
ray.init()
time.sleep(2)

2019-07-02 23:40:21,535	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-02_23-40-21_534793_25689/logs.
2019-07-02 23:40:21,642	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:18998 to respond...
2019-07-02 23:40:21,754	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:32819 to respond...
2019-07-02 23:40:21,760	INFO services.py:806 -- Starting Redis shard with 1.64 GB max memory.
2019-07-02 23:40:21,791	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-02_23-40-21_534793_25689/logs.
2019-07-02 23:40:21,795	INFO services.py:1442 -- Starting the Plasma object store with 2.47 GB memory using /dev/shm.


# TEST

In [105]:
%%time
# parallel version
@ray.remote
def run2():  # same as run1 
    for i in range(100000):
        env = gym.make("CartPole-v0")
        env.reset()
        steps = []
        for _ in range(100):
            obs, reward, done, info = env.step(env.action_space.sample())
            if(done):
                break
    return len(steps)

# note: maybe run this twice to warmup the system
for i in range(1):
    result = ray.get([run2.remote() for i in range(7)])

Exception in thread ray_import_thread:
Traceback (most recent call last):
  File "/home/nishantkr18/.local/lib/python3.5/site-packages/redis/connection.py", line 185, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.5/threading.py", line 862, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nishantkr18/.local/lib/python3.5/site-packages/ray/import_thread.py", line 69, in _run
    msg = import_pubsub_client.get_message()
  File "/home/nishantkr18/.local/lib/python3.5/site-packages/redis/client.py", line 3135, in get_message
    response = self.parse_response(block=False, timeout=timeout)
  File "/home/nishantkr18/.local/lib/python3.5/site-packages/redis/client.py", line 3036, in parse_respons

KeyboardInterrupt: 

In [87]:
%%time
def run2():  # same as run1 
    for i in range(10000):
        env = gym.make("CartPole-v0")
        env.reset()
        steps = []
        for _ in range(100):
            obs, reward, done, info = env.step(env.action_space.sample())
            if(done):
                break
    return len(steps)

# note: maybe run this twice to warmup the system
for i in range(10):
    result = run2()

CPU times: user 46.4 s, sys: 20.1 s, total: 1min 6s
Wall time: 58.5 s


In [22]:
%%time
@ray.remote
def f(x):
    for i in range(10000):
        sum(range(i*100, 400010))
#     time.sleep(4)
    return x

# Start 4 tasks in parallel.
result_ids = []
for i in range(4):
    result_ids.append(f.remote(i))

print(len(result_ids))
# Wait for the tasks to complete and retrieve the results.
# With at least 4 cores, this will take 1 second.
%time results = ray.get(result_ids)  # [0, 1, 2, 3]

4
CPU times: user 3.46 ms, sys: 0 ns, total: 3.46 ms
Wall time: 90.6 ms
[0, 2, 4, 6]
CPU times: user 6.86 ms, sys: 0 ns, total: 6.86 ms
Wall time: 93.1 ms


In [ ]:
%%time
envS = []
population = 100000
import time
for i in range(population):
    name = 'MountainCar-v0'
    name = 'CarRacing-v0'
    name = 'BipedalWalker-v2'
    name = 'CartPole-v0'
    envS.append(gym.make(name))
    envS[i].reset()

for env in envS:    
    for _ in range(100):
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        if(done):
            break
#         print(state, reward, done, info)
#         time.sleep(0.1)
#         clear_output()

        if(population == 1):
            env.render()
for env in envS:
    env.close()

In [15]:
%%time

envS = []
population = 100000
import time
for i in range(population):
    name = 'MountainCar-v0'
    name = 'CarRacing-v0'
    name = 'BipedalWalker-v2'
    name = 'CartPole-v0'
    envS.append(gym.make(name))
    envS[i].reset()

for env in envS:    
    for _ in range(100):
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        if(done):
            break
#         print(state, reward, done, info)
#         time.sleep(0.1)
#         clear_output()

        if(population == 1):
            env.render()
for env in envS:
    env.close()

2019-07-01 14:50:31,458	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-01_14-50-31_458190_30156/logs.
2019-07-01 14:50:31,626	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:59063 to respond...


CPU times: user 6.36 ms, sys: 0 ns, total: 6.36 ms
Wall time: 40.3 ms


2019-07-01 14:50:31,825	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:26617 to respond...
2019-07-01 14:50:31,835	INFO services.py:806 -- Starting Redis shard with 1.64 GB max memory.
2019-07-01 14:50:31,979	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-07-01_14-50-31_458190_30156/logs.
2019-07-01 14:50:31,981	INFO services.py:1442 -- Starting the Plasma object store with 2.47 GB memory using /dev/shm.


CPU times: user 40.4 s, sys: 19.1 s, total: 59.4 s
Wall time: 57 s
